In [ ]:
#This notebook reproduces the graphs shown in Figure 9 of the paper (doi.org/10.31224/2422)
'''
Demonstrated features:
1. Running `compute_geochemical_consensus` with cfg['handle_breakdown']
   enabled. This activates split-sequence analysis which seeks to
   overcome MCD estimator breakdown when the samples contain an unknown
   degree of contamination (i.e. up to 50% of outliers).
2. Validation is performed using multiprocessing (N cpu cores)
3. Using tools from "custom_graphics.py" to display data in a ternary diagram
   - `ternary_coords` converts a 3D composition vector to 2D canvas coordinates
   - `draw_ternary_borders` produces the basic layout of a ternary diagram.
''';

__copyright__ = "Copyright (c) 2022, Raymond Leung"
__license__   = "GPLv3"

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import multiprocessing
import time
import validation

from matplotlib import cm, pyplot as plt

In [ ]:
validation_data_file = 'data/synthesized_samples.zip'
df = pd.read_csv(validation_data_file, compression='zip', header=0, sep=',')

In [ ]:
#input data inspection
df.loc[[27340,49000,252100]]

In [ ]:
#config parameters
cfg = {'modelled_elements': ['Fe','SiO2','Al2O3'],
       'chemical_weights': [0.5,0.325,0.175],
       'handle_breakdown': True,
       'terminate_early': True,
       'sample_size': 32}

In [ ]:
#perform validation using multiprocessing
mp = validation.Multiprocessor()
num_processes = multiprocessing.cpu_count()
sample_size = cfg['sample_size']
t0 = time.time()

for i in range(num_processes):
    mp.run(validation.worker, i, num_processes, df, cfg)

ret = mp.wait()
t1 = time.time()
assert len(ret) == num_processes
print('compute_geochemical_consensus completed in {}s '
      'using {} cores'.format(t1 - t0, num_processes))

#each element in `ret` corresponds to [process_num, L] where L represents
#a vertical stack of many [eta_outliers, d_gmean, d_unmasked, f_consensus]

#the results may arrive out-of-order, so we need to rearrange
#them if necessary to match the input
order = np.argsort([r[0] for r in ret])
assessment = np.vstack([ret[i][1] for i in order])

In [ ]:
#graphical analysis
parameters = ['mode(phi)', 'noise(nu)', 'cluster_dist(delta)', 'frac_outliers(eta)']
X = np.hstack([df.loc[:,parameters].values[::sample_size], assessment])

show_errorbar = True
iMode, iNu, iDelta, iEta = 0, 1, 2, 3
iConsensus = X.shape[1] - 1
v_delta = np.linspace(0,1,21)[1:-1]
v_eta = np.linspace(0,1,17)
stepsize = 0.5 / (len(v_eta)-1)
midgray = [0.7,0.7,0.7]
handle_breakdown = cfg['handle_breakdown']
prefer_loc = 'lower center' if handle_breakdown else 'center'
prefer_ncol = 10 if handle_breakdown else 1
prefer_colsp = 1

for mode in [0,1]:
    for nu in [0.025]:
        plt.figure(figsize=(10,6))
        vmean = {}
        hdl = []
        for i, delta in enumerate(v_delta):
            vmean[delta] = []
            rgb = cm.tab20c(i) 
            for eta in v_eta:
                idx = np.where((X[:,iMode]==mode) & np.isclose(X[:,iNu],nu) &
                               np.isclose(X[:,iDelta],delta) & np.isclose(X[:,iEta],eta))[0]
                scores = X[idx,iConsensus]
                sigma = np.std(scores)
                low, high = np.percentile(scores,25), np.percentile(scores,75)
                scores = scores[(scores >= low) & (scores <= high)]
                mu = np.mean(scores)
                stderr = sigma / np.sqrt(len(scores))
                if not handle_breakdown and (eta > 0.4 and eta < 0.6):
                    mu *= np.nan
                    sigma *= np.nan
                vmean[delta].append(mu)
                if show_errorbar and (handle_breakdown or eta <= 0.4 or eta >= 0.6):
                    plt.plot(eta, mu, 'ko', ms=5)
                    plt.plot([eta]*2, [mu-stderr,mu+stderr], ls='-', lw=0.5, c=midgray)
                    plt.plot([eta-0.3*stepsize, eta+0.3*stepsize],
                             [mu-stderr]*2, ls='-', lw=0.5, c=midgray)
                    plt.plot([eta-0.3*stepsize, eta+0.3*stepsize],
                             [mu+stderr]*2, ls='-', lw=0.5, c=midgray)
            h = plt.plot(v_eta, vmean[delta], '-', c=rgb, label='%.3g' % delta)
            hdl.append(h)
        plt.xlabel(r'$\eta$')
        plt.ylabel(r'$f_{\mathrm{consensus}}$')
        ax = plt.gca()
        ax.set_xlim([-0.5*stepsize, 1+0.5*stepsize])
        ax.set_ylim([0.45, 1])
        plt.title(r'mode=$\phi_{}$, $\nu$={}'.format(mode, '%.3g' % nu))
        plt.legend(title=r'$\delta$', fontsize='small', loc=prefer_loc,
                   ncol=prefer_ncol, columnspacing=prefer_colsp)
        plt.show()

In [ ]:
#Methods for plotting 3D composition vectors in 2D ternary diagram
from custom_graphs import ternary_coords, draw_ternary_borders
from matplotlib import pyplot as plt

test_enabled = True
if test_enabled:
    composition_vects = [(.35,.55,.1), (.8,.15,.05)]
    txy = np.array([ternary_coords(*c) for c in composition_vects])
    draw_ternary_borders()
    plt.plot(txy[:,0], txy[:,1], 'o')